Dynamics = Time-dependent Elasticity
===

In [1]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw

In [2]:
shape = MoveTo(0,-0.05).Rectangle(1,0.1).Face()
shape = shape + Circle((1,0), 0.15).Face()
shape.edges.Min(X).name="left"
shape.edges.Min(X).maxh=0.01

mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.05)).Curve(3)

Draw (mesh);

WebGuiWidget(value={'ngsolve_version': '6.2.2105-170-ge8fcd5755', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

Cauchy-Green tensor and hyperelastic energy density:

In [3]:
E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def C(u):
    F = Id(2) + Grad(u)
    return F.trans * F

def NeoHooke (C):
    return 0.5*mu*(Trace(C-Id(2)) + 2*mu/lam*Det(C)**(-lam/2/mu)-1)

energy functional
$$
J(u) = \int W(C(u)) - f u
$$

equilibrium of forces in the time-independent (= stationary) case:

$$
\left< J^\prime(u), v \right> = 0 \qquad \forall \, v
$$

non-equilibrium leads to acceleration (Newton's law):

$$
\int \rho \ddot u v = -\left< J^\prime(u), v \right> \qquad \forall \, v
$$

with acceleration $a := \ddot u$, discretized Newton's law becomes

$$
M a = - J^\prime(u)
$$

for displacement vector $u$, acceleration vector $a$, and mass matrix $M$

## Newmark timestepping 
is trapezoidal rule for displacement $u$ and velocity $v$:

\begin{eqnarray*}
\frac{\hat u- u}{\tau} = \tfrac{1}{2} (v + \hat v) \\
\frac{\hat v- v}{\tau} = \tfrac{1}{2} (a + \hat a) \\
\end{eqnarray*}

leads to

$$
\hat u = u + \tau v + \frac{\tau^2}{4} (a + \hat a),
$$

or making $\hat a$ explizit to 

$$
\hat a = \frac{4}{\tau^2} (\hat u - u - \tau v) - a
$$

Inserting Newton's law for the new time-step

$$
M (\tfrac{4}{\tau^2} (\hat u - u - \tau v) - a) = - J^\prime(\hat u)
$$

leads to the equation for the new displacement $\hat u$.

In [4]:
#fes = H1(mesh, order=4, dirichlet="left", dim=mesh.dim)
# fes = H1(mesh, order=4, dim=mesh.dim)
fes = VectorH1(mesh, order=4)

#fes_f = NumberSpace(mesh, dim=mesh.dim) 
fes_f = NumberSpace(mesh) * NumberSpace(mesh)
# Lagrange parameter space for enforcing hinge 
# (constraint: phi is rigid SO(2) rotation)
#fes_f = NumberSpace(mesh) * NumberSpace(mesh) * NumberSpace(mesh) * NumberSpace(mesh) 

X = fes * fes_f
# X = fes * NumberSpace(mesh) * NumberSpace(mesh)


# fx, fy = fes_f.TrialFunction()
# fxt, fyt = fes_f.TestFunction()

gf = GridFunction(X)

# gfu = GridFunction(fes)
gfu = gf.components[0]

gfvel = GridFunction(fes)
gfacc = GridFunction(fes)

# gfunew = GridFunction(fes)
gfnew = GridFunction(X)

gfunew = gfnew.components[0]

gfaccnew = GridFunction(fes)

In [5]:
# x_CF = CF((x,y))
# Id_mat = CF(((1,0),(0,1)))

# o_cond = (OuterProduct(u + x_CF, u + x_CF) - Id_mat)

# o_cond[0,0]*fxx*ds("left")
# o_cond[0,1]*fxy*ds("left")
# o_cond[1,0]*fyx*ds("left")
# o_cond[1,1]*fyy*ds("left")


# (OuterProduct(u + x_CF, u + x_CF) - Id_mat)*(fx, fy)*ds("left")
# o_cond = (OuterProduct(u + x_CF, u + x_CF) - Id_mat)
# o_cond = (OuterProduct(grad(u) + Id_mat, grad(u) + Id_mat) - Id_mat)

# (u*u.trans + x_CF*u.trans + u*x_CF.trans + x_CF*x_CF.trans )*ds("left")

In [6]:
rho = 0.1
force = CF( (0,-rho*9.81) )
tau = 0.01

# u,v = fes.TnT()
u,fx,fy = X.TrialFunction()
v,fxt, fyt = X.TestFunction()

# u, fxx, fxy, fyx, fyy = X.TrialFunction()
# v, fxxt, fxyt, fyxt, fyyt = X.TestFunction()

acc = 4/(tau*tau)*(u-gfu-tau*gfvel) - gfacc

#a = BilinearForm(fes)
a = BilinearForm(X)
a += rho*acc*v*dx
a += Variation(NeoHooke(C(u))*dx)
a += Variation(-force*u*dx)

#phi = x + u
# phi should be a rigid body motion on the left border:
# (x+u)(x+u)^T = 1, det(x + u) = 1 ??
# a += Variation((OuterProduct(u + x_CF, u + x_CF) - Id_mat)*(fx, fy)*ds("left"))

# x_CF = CF((x,y))
# Id_mat = CF(((1,0),(0,1)))
# o_cond = grad(u)*grad(u).trans + grad(u) + grad(u).trans

# # Trace for gradient not defined because H1?
# o_lagrange = o_cond[0,0]*fxx*ds("left")
# o_lagrange += o_cond[0,1]*fxy*ds("left")
# o_lagrange += o_cond[1,0]*fyx*ds("left")
# o_lagrange += o_cond[1,1]*fyy*ds("left")
# a += Variation(o_lagrange)

# average force on hinge vanishes (replacement for dirichlet BC???)
a += (1/0.1)*(u*CF((fxt,fyt)) + v*CF((fx,fy))) * ds("left")

a simple Newton solver, using automatic differentiation for residual and tangential stiffness:

In [7]:
# def SolveNewton(gfu):
def SolveNewton(gf):
#     res = gfu.vec.CreateVector()
#     w = gfu.vec.CreateVector()
    
    res = gf.vec.CreateVector()
    w = gf.vec.CreateVector()
    
    for it in range(10):
#         a.Apply(gfu.vec, res)
#         a.AssembleLinearization(gfu.vec)
        a.Apply(gf.vec, res)
        a.AssembleLinearization(gf.vec)
        
        inv = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky") 
        w.data = inv*res
#         gfu.vec.data -= w
        gf.vec.data -= w
        
        # print ("it", it, "Norm w = ", Norm(w))

In [8]:
scene = Draw (C(gfunew)[0,0], mesh, deformation=gfu)

res = gf.vec.CreateVector()
w = gf.vec.CreateVector()

a.Apply(gf.vec, res)
a.AssembleLinearization(gf.vec)

inv = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky") 
w.data = inv*res

gf.vec.data -= w

WebGuiWidget(value={'ngsolve_version': '6.2.2105-170-ge8fcd5755', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…

In [9]:
scene.Redraw()

In [ ]:
# scene = Draw (C(gfunew)[0,0], mesh, deformation=gfu)

scene = Draw (C(gfunew)[0,0], mesh, deformation=gfu)

t = 0
tend = 10
while t < tend:
#     SolveNewton(gfunew)
    SolveNewton(gfnew)
    
    gfaccnew.vec.data = 4/tau**2 * (gfunew.vec-gfu.vec-tau*gfvel.vec) \
        - gfacc.vec
        
    gfvel.vec.data += tau/2 * (gfacc.vec + gfaccnew.vec)
    gfu.vec.data = gfunew.vec
    gfacc.vec.data = gfaccnew.vec
    
    t = t+tau
    scene.Redraw()

WebGuiWidget(value={'ngsolve_version': '6.2.2105-170-ge8fcd5755', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, '…